In [437]:
import Ipynb_importer
import matplotlib.pyplot as plt
import MUSIC
import numpy as np
from scipy import signal
from numpy import pi
from numpy.linalg import cholesky
from numpy.linalg import eig
from numpy.linalg import matrix_rank

In [439]:
theta_S = np.array([-10, 0, 10])
snapshots = 200
# M = 8
SNR = 30

N = 1000

STD_RMES_list = []
SS_RMES_list = []
for M in range(5, 20):
    RMES = 0
    
    for itr in range(N):
        theta, P_mu = MUSIC.STD_MUSIC(SNR, snapshots, M)
        peakIdx = signal.argrelmax(abs(P_mu))
        ii = np.where(P_mu[peakIdx[0]] > P_mu.mean())
        peakIdx = peakIdx[0][ii]
        theta_h = theta[peakIdx]    # theta hat
        if len(theta_S) > len(theta_h):
            theta_h = np.hstack((theta_h, np.zeros(len(theta_S) - len(theta_h))))
            theta_h.sort()
        else:
            theta_S = np.hstack((theta_S, np.zeros(len(theta_h) - len(theta_S))))
            theta_S.sort()
        
        res = theta_S - theta_h
        RMES = RMES + np.sqrt(np.dot(res.T, res)/len(theta_S))
        
    RMES = RMES/N
    STD_RMES_list.append(RMES)
    
    RMES = 0
    
    for itr in range(N):
        theta, P_mu = MUSIC.SS_MUSIC(SNR, snapshots, M)
        peakIdx = signal.argrelmax(abs(P_mu))
        ii = np.where(P_mu[peakIdx[0]] > P_mu.mean())
        peakIdx = peakIdx[0][ii]
        theta_h = theta[peakIdx]    # theta hat
        if len(theta_S) > len(theta_h):
            theta_h = np.hstack((theta_h, np.zeros(len(theta_S) - len(theta_h))))
            theta_h.sort()
        else:
            theta_S = np.hstack((theta_S, np.zeros(len(theta_h) - len(theta_S))))
            theta_S.sort()
        
        res = theta_S - theta_h
        RMES = RMES + np.sqrt(np.dot(res.T, res)/len(theta_S))
        
    RMES = RMES/N
    SS_RMES_list.append(RMES)

STD_RMES_list = np.array(STD_RMES_list)
SS_RMES_list = np.array(SS_RMES_list)

fig = plt.figure(0)
plt.plot(range(5, 20), STD_RMES_list, label='MUSIC', marker='o', linestyle='-')
plt.hold
plt.plot(range(5, 20), SS_RMES_list, label='SS-MUSIC', marker='*', linestyle='--')
plt.legend()
plt.xlabel('The Number of Sensor')
plt.ylabel('RMES')
plt.title('The number of test: ' + str(N))
fig.set_size_inches(10, 10)
plt.show()
fig.savefig('Two mathods in different M')